In [1]:
# pip install CensusData

In [2]:
import pandas as pd
import censusdata
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import glob

%matplotlib inline
%config InlineBackend.figure_format ='retina'

In [3]:
sharedFolder = '/work/group/egodat/reu23_clark/'
dataFolder = sharedFolder + 'data/'
acsYears = [2010, 2015, 2019, 2021]

In [4]:
codebook = pd.read_excel("Codebook.xlsx")
acsCodes = codebook[codebook['data source'] == 'ACS'].copy()

acsCodes['firstYear'] = acsCodes['year(s)'].apply(lambda x: int(x.split(',')[0]))

demographic_variables_dict = {
    r['var code']: (r['column name'], r['firstYear'], r['agg type']) for _, r in acsCodes.iterrows()
}
demographic_variables_dict

{'B01003_001E': ('Total_Population', 2010, 'sum'),
 'B01001_002E': ('Total_Male_Pop', 2010, 'sum'),
 'B01001_026E': ('Total_Female_Pop', 2010, 'sum'),
 'B01002_001E': ('Median_Age', 2010, 'mean'),
 'B01002_002E': ('M_Median_Age', 2010, 'mean'),
 'B01002_003E': ('F_Median_Age', 2010, 'mean'),
 'B02001_002E': ('Race_white', 2010, 'sum'),
 'B02001_003E': ('Race_black', 2010, 'sum'),
 'B02001_004E': ('Race_Am_Indian', 2010, 'sum'),
 'B02001_005E': ('Race_Asian', 2010, 'sum'),
 'B02001_006E': ('Race_Pac_Isl', 2010, 'sum'),
 'B20004_001E': ('Median_Income', 2010, 'mean'),
 'B20004_002E': ('Less_High', 2010, 'mean'),
 'B20004_003E': ('High_Equiv', 2010, 'mean'),
 'B20004_004E': ('College_Assoc_Equiv', 2010, 'mean'),
 'B20004_005E': ('Bachelors', 2010, 'mean'),
 'B20004_006E': ('Grad_Prof', 2010, 'mean'),
 'B24031_002E': ('Ag_For_Fish_Hunt_Mine', 2010, 'mean'),
 'B24031_005E': ('Construction', 2010, 'mean'),
 'B24031_006E': ('Manufacturing', 2010, 'mean'),
 'B24031_007E': ('Wholesale', 2010, '

In [5]:
statesShp = geopandas.read_file(dataFolder + "/cb_2018_us_state_500k.zip").to_crs(3857)
statesShp = statesShp[['GEOID', 'STUSPS', 'NAME', 'geometry']]

def plotUS():
    ax = statesShp.plot(figsize=(20, 10))
    ax.axis('equal')
    plt.xlim(-1.5e7, -0.7e7)
    plt.ylim(2.5e6, 7e6)
    return ax

In [6]:
xwalk = pd.read_pickle(dataFolder + "/xwalk_data_combined.pkl")
xwalk

,ctyname,bgrp,cbsa
0,"Rockingham County, NH",330150710012,14460
1,"Merrimack County, NH",330130380002,18180
2,"Merrimack County, NH",330130415001,18180
3,"Hillsborough County, NH",330110225011,31700
4,"Hillsborough County, NH",330110103022,31700
...,...,...,...
57404,"Elko County, NV",320079515003,21220
57405,"Elko County, NV",320079515003,21220
57406,"Elko County, NV",320079517002,21220
57407,"Elko County, NV",320079502001,21220


In [7]:
# https://api.census.gov/data/2021/acs/acs5/variables.html

# variable_code: (readable_name, earliest_year, aggrigation_type)

# B20004_001E is for median income by educational attainment

dfs = []

for year in acsYears:
    df = censusdata.download(
        'acs5',
        # 'acs3',
        year,
        censusdata.censusgeo([('county', '*')]),
        [k for k, (n, y, a) in demographic_variables_dict.items() if y <= year] #Skip if not available for year
    )
    df['year'] = year
    dfs.append(df)

data = pd.concat(dfs)
data

,B01003_001E,B01001_002E,B01001_026E,B01002_001E,B01002_002E,B01002_003E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,...,B08121_007E,year,B24114_001E,B24114_064E,B24114_068E,B24114_065E,B24114_066E,B24114_067E,B24114_069E,B24114_070E
"Manatí Municipio, Puerto Rico: Summary level: 050, state:72> county:091",44631,21297,23334,35.8,34.4,37.5,38275,1871,46,11,...,8833.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Merced County, California: Summary level: 050, state:06> county:047",250699,126197,124502,29.3,28.4,30.3,166042,9781,2533,18668,...,17769.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Modoc County, California: Summary level: 050, state:06> county:049",9605,4816,4789,45.4,42.9,46.5,8470,96,345,124,...,16379.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Mono County, California: Summary level: 050, state:06> county:051",13905,7630,6275,36.5,34.8,38.8,11368,118,674,37,...,29633.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Monterey County, California: Summary level: 050, state:06> county:053",407435,209831,197604,32.8,31.9,33.7,284397,13091,3612,25553,...,23391.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Vega Baja Municipio, Puerto Rico: Summary level: 050, state:72> county:145",54544,26057,28487,43.1,41.5,44.6,30690,2409,50,9,...,14500.0,2021,None,None,None,None,None,None,None,None
"Vieques Municipio, Puerto Rico: Summary level: 050, state:72> county:147",8317,4239,4078,43.6,40.9,45.9,2592,629,0,14,...,20771.0,2021,None,None,None,None,None,None,None,None
"Villalba Municipio, Puerto Rico: Summary level: 050, state:72> county:149",22341,10796,11545,42.0,40.7,42.9,10502,1784,0,0,...,-666666666.0,2021,None,None,None,None,None,None,None,None
"Yabucoa Municipio, Puerto Rico: Summary level: 050, state:72> county:151",31047,15000,16047,44.9,43.7,46.0,3190,16119,0,11,...,-666666666.0,2021,None,None,None,None,None,None,None,None


In [8]:
#Extract block group code
xwalk['fips'] = (xwalk['bgrp'] / 10**7).astype(int)
fipsToCbsa = xwalk.groupby('fips').first()['cbsa']
fipsToCbsa = dict(zip(fipsToCbsa.index, fipsToCbsa.values)) #Dict is faster for lookup

In [9]:
# to rename variable, add to dict
data.fillna(value=np.nan, inplace=True)
data.rename(columns={k: n for k, (n, y, a) in demographic_variables_dict.items()},
            inplace=True)
data = data.reset_index().rename(columns = {"index":"FIPS"})
data["FIPS"] = data["FIPS"].apply(lambda x: x.params()[0][1] + x.params()[1][1])
data['FIPS'] = data['FIPS'].astype(int)
data[data < 0] = np.nan
data

,FIPS,Total_Population,Total_Male_Pop,Total_Female_Pop,Median_Age,M_Median_Age,F_Median_Age,Race_white,Race_black,Race_Am_Indian,...,Median_Income_Worked_home,year,Num_Total_Worker,Num_Comp_Info_Res,Num_Soft_Dev,Num_Comp_Sys_Analyst,Num_Info_Sec_Analyst,Num_Comp_Programmer,Num_Soft_Qual,Num_Web_Dev
0,72091,44631,21297,23334,35.8,34.4,37.5,38275,1871,46,...,8833.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6047,250699,126197,124502,29.3,28.4,30.3,166042,9781,2533,...,17769.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6049,9605,4816,4789,45.4,42.9,46.5,8470,96,345,...,16379.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6051,13905,7630,6275,36.5,34.8,38.8,11368,118,674,...,29633.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6053,407435,209831,197604,32.8,31.9,33.7,284397,13091,3612,...,23391.0,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12877,72145,54544,26057,28487,43.1,41.5,44.6,30690,2409,50,...,14500.0,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12878,72147,8317,4239,4078,43.6,40.9,45.9,2592,629,0,...,20771.0,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12879,72149,22341,10796,11545,42.0,40.7,42.9,10502,1784,0,...,NaN,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12880,72151,31047,15000,16047,44.9,43.7,46.0,3190,16119,0,...,NaN,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data['CBSA'] = data['FIPS'].apply(lambda c: fipsToCbsa.get(c, 99999))
data

,FIPS,Total_Population,Total_Male_Pop,Total_Female_Pop,Median_Age,M_Median_Age,F_Median_Age,Race_white,Race_black,Race_Am_Indian,...,year,Num_Total_Worker,Num_Comp_Info_Res,Num_Soft_Dev,Num_Comp_Sys_Analyst,Num_Info_Sec_Analyst,Num_Comp_Programmer,Num_Soft_Qual,Num_Web_Dev,CBSA
0,72091,44631,21297,23334,35.8,34.4,37.5,38275,1871,46,...,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99999
1,6047,250699,126197,124502,29.3,28.4,30.3,166042,9781,2533,...,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32900
2,6049,9605,4816,4789,45.4,42.9,46.5,8470,96,345,...,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99999
3,6051,13905,7630,6275,36.5,34.8,38.8,11368,118,674,...,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99999
4,6053,407435,209831,197604,32.8,31.9,33.7,284397,13091,3612,...,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12877,72145,54544,26057,28487,43.1,41.5,44.6,30690,2409,50,...,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99999
12878,72147,8317,4239,4078,43.6,40.9,45.9,2592,629,0,...,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99999
12879,72149,22341,10796,11545,42.0,40.7,42.9,10502,1784,0,...,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99999
12880,72151,31047,15000,16047,44.9,43.7,46.0,3190,16119,0,...,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99999


In [11]:
aggType = {n: a for k, (n, y, a) in demographic_variables_dict.items()}

#Add derived columns here for county level calculations
data['Pop_BS_Above'] = data['Pop_Bachelors'] + data['Pop_Grad']
aggType['Pop_BS_Above'] = 'sum'
data['Pop_HS_Above'] = data['Pop_HS_Grad'] + data['Pop_Some_College'] + data['Pop_BS_Above']
aggType['Pop_HS_Above'] = 'sum'

dataByCbsa = data.groupby(by=['CBSA', 'year']).agg(aggType).reset_index()

In [12]:
#Add derived columns here for CBSA level calculations
dataByCbsa['Pct_Less_HS'] = dataByCbsa['Pop_Less_HS'] / dataByCbsa['Education_Pop']
dataByCbsa['Pct_HS_Grad'] = dataByCbsa['Pop_HS_Grad'] / dataByCbsa['Education_Pop']
dataByCbsa['Pct_Some_College'] = dataByCbsa['Pop_Some_College'] / dataByCbsa['Education_Pop']
dataByCbsa['Pct_Bachelors'] = dataByCbsa['Pop_Bachelors'] / dataByCbsa['Education_Pop']
dataByCbsa['Pct_Grad'] = dataByCbsa['Pop_Grad'] / dataByCbsa['Education_Pop']
dataByCbsa['Pct_BS_Above'] = dataByCbsa['Pop_BS_Above'] / dataByCbsa['Education_Pop']
dataByCbsa['Pct_HS_Above'] = dataByCbsa['Pop_HS_Above'] / dataByCbsa['Education_Pop']

In [13]:
dataByCbsa.to_pickle(sharedFolder + 'ACS_data.pkl')